In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import meep as mp
import gdsfactory.simulation.modes as gm

# MPB modes

MPB has periodic boundary conditions.

## Find modes

### Strip waveguides

Strip waveguides are fully etch and don't have a slab

In [ ]:
mode_solver = gm.get_mode_solver_rib(wg_width=0.4, ncore=3.47, nclad=1.44, wg_thickness=.22, resolution=20, sy=3, sz=3, nmodes=4)
modes = gm.find_modes(mode_solver=mode_solver, parity=mp.NO_PARITY)
m1 = modes[1]
m2 = modes[2]
m3 = modes[3]

In [ ]:
m1.plot_eps()

In [ ]:
m1.plot_ex()

In [ ]:
m1.plot_e_all()

In [ ]:
m1.plot_e()

In [ ]:
m2.plot_e()

In [ ]:
m3.plot_e() # not guided

In [ ]:
m1.neff

In [ ]:
m2.neff

In [ ]:
# the third mode does not propagate and its neff is below the cladding index
m3.neff

### Rib waveguides

Ribe waveguides have a slab (not fully etched)

In [ ]:
import gdsfactory.simulation.modes as gm
import meep as mp

mode_solver = gm.get_mode_solver_rib(slab_thickness=90e-3)
modes = gm.find_modes(mode_solver=mode_solver, mode_number=1, nmodes=2, resolution=10)
m1 = modes[1]
m2 = modes[2]

In [ ]:
eps = mode_solver.get_epsilon()

In [ ]:
m1.plot_eps()

In [ ]:
m1.plot_e()

## Symmetries

You can exploit symmetries to reduce computation time as well as finding only (TE or TM) modes

asuming propagating in X direction

- TE: mp.ODD_Y + mp.EVEN_Z
- TM: mp.EVEN+Y + mp.ODD_Z, all energy in z component

### TM: mp.ODD_Y + mp.EVEN_Z

You can define an even Y parity to find only the TM modes

In [ ]:
mode_solver = gm.get_mode_solver_rib(wg_width=1.0, ncore=3.47, nclad=1.44, wg_thickness=.22, resolution=32, sy=6, sz=6)
modes = gm.find_modes(mode_solver=mode_solver, mode_number=1, parity= mp.EVEN_Y+mp.ODD_Z, nmodes=2)
m1 = modes[1]
m2 = modes[2]

In [ ]:
m1.plot_e()

In [ ]:
mode_solver = gm.get_mode_solver_rib(wg_width=1.0, ncore=3.47, nclad=1.44, wg_thickness=.22, resolution=32, sz=6, sy=6)
modes = gm.find_modes(mode_solver=mode_solver, mode_number=1, parity= mp.EVEN_Y+mp.ODD_Z, nmodes=2)
m1 = modes[1]
m2 = modes[2]

In [ ]:
modes.keys()

In [ ]:
m1.plot_e()

### ODD_Y (TE)

In [ ]:
mode_solver = gm.get_mode_solver_rib(wg_width=0.20, ncore=3.47, nclad=1.44, wg_thickness=.22, resolution=32, sy=6, sz=6)
modes = gm.find_modes(mode_solver=mode_solver, mode_number=1, parity=mp.ODD_Y, nmodes=2)
m1 = modes[1]
m2 = modes[2]

In [ ]:
m1.plot_e()

## Sweep waveguide width

### Strip

In [ ]:

df = gm.find_neff_vs_width(filepath='neff_vs_width.csv')
df

In [ ]:
gm.plot_neff_vs_width(df)

### Rib waveguides

In [ ]:
import matplotlib.pyplot as plt
import gdsfactory.simulation.modes as gm
import meep as mp

mode_solver = gm.get_mode_solver_rib(wg_width=0.4, ncore=3.47, nclad=1.44, wg_thickness=220e-3, resolution=32, sz=6, sy=6, nmodes=4, slab_thickness=90e-3)
modes = gm.find_modes(mode_solver=mode_solver, parity=mp.NO_PARITY)
m1 = modes[1]
m2 = modes[2]
m3 = modes[3]

In [ ]:
m1.plot_eps()
m1.neff

In [ ]:
m1.plot_e()
m1.neff

In [ ]:
m2.plot_e()
m2.neff

In [ ]:
ws = gm.find_neff_vs_width(slab_thickness=90e-3, filepath='neff_vs_width_rib.csv')
ws.plot()

### Nitride

In [ ]:
mode_solver = gm.get_mode_solver_rib(wg_width=1.0, ncore=2., nclad=1.44, wg_thickness=400e-3, sz=6, sy=10, nmodes=4)
modes = gm.find_modes(mode_solver=mode_solver, parity=mp.NO_PARITY)
m1 = modes[1]
m2 = modes[2]
m3 = modes[3]

In [ ]:
m1.plot_eps()

In [ ]:
m1.plot_ey()

In [ ]:
m2.plot_ex()

In [ ]:
m3.plot_ey()

In [ ]:
ws = gm.find_neff_vs_width(w2=1.2, wg_thickness=0.4, ncore=2., sy=10., filepath='neff_vs_width_nitride.csv')

In [ ]:
ws.plot()

## Convergence tests

Before launching a set of simulations you need to make sure you have the correct simulation settings:

- resolution: resolution
- sx: Size of the simulation region in the x-direction (default=4.0)
- sy: Size of the simulation region in the y-direction (default=4.0)


In [ ]:
import gdsfactory.simulation.modes as gm
import numpy as np
import matplotlib.pyplot as plt

resolutions = np.linspace(10, 50, 5)

neffs = []
ngs= []

for resolution in resolutions:
    modes = gm.find_modes(wg_width=0.5, ncore=3.5, nclad=1.44, wg_thickness=.22, resolution=resolution)
    mode = modes[1]
    ngs.append(mode.ng)
    neffs.append(mode.neff)

In [ ]:
plt.plot(resolutions, ngs, 'o-')
plt.ylabel('ng')
plt.xlabel('resolution (pixels/um)')

In [ ]:
plt.plot(resolutions, neffs, 'o-')
plt.ylabel('neff')
plt.xlabel('resolution (pixels/um)')

In [ ]:
szs = np.linspace(4, 6, 6)
neffs = []
ngs= []

for sz in szs:
    modes = gm.find_modes(
        wg_width=0.5, ncore=3.5, nclad=1.44, wg_thickness=.22, resolution=20, sz=sz
    )
    mode = modes[1]
    ngs.append(mode.ng)
    neffs.append(mode.neff)

In [ ]:
plt.plot(szs, neffs, 'o-')
plt.ylabel('neff')
plt.xlabel('simulation size in z(um)')

In [ ]:
sys = np.linspace(2, 6, 6)
neffs = []
ngs= []

for sy in sys:
    modes = gm.find_modes(
        wg_width=0.5, ncore=3.5, nclad=1.44, wg_thickness=.22, resolution=20, sy=sy
    )
    mode = modes[1]
    ngs.append(mode.ng)
    neffs.append(mode.neff)

In [ ]:
plt.plot(sys, neffs, 'o-')
plt.ylabel('neff')
plt.xlabel('simulation size in y (um)')